# Position and Portfolio Value: Calculating What You Own

**Time:** 12 minutes

**Prerequisites:** 01_stock_basics.ipynb

---

Now that you understand individual stocks, let's learn how to calculate the value of what you own. This is fundamental to portfolio management and the first thing any investment AI agent needs to do.

## Position: Your Stake in a Single Stock

A **position** is your holding in a single investment.

**Position Value Formula:**

$$\text{Position Value} = \text{Number of Shares} \times \text{Current Price}$$

**Example:**
- You own 100 shares of Apple (AAPL)
- AAPL is trading at $175
- Your position value = 100 × $175 = **$17,500**

**Mental Model:** Think of it like counting your pizza slices. Each slice has a value (the current price), and you multiply by how many slices you have.

In [ ]:
# Install required packages
!pip install -q yfinance pandas

In [ ]:
import yfinance as yf

def get_price(ticker):
    """Get current price for a ticker."""
    stock = yf.Ticker(ticker)
    info = stock.info
    return info.get('currentPrice') or info.get('regularMarketPrice') or info.get('previousClose')

def calculate_position_value(ticker, shares):
    """Calculate the dollar value of a position.

    Args:
        ticker: Stock symbol (e.g., 'AAPL')
        shares: Number of shares owned

    Returns:
        Position value in dollars
    """
    price = get_price(ticker)
    value = shares * price
    return {
        'ticker': ticker,
        'shares': shares,
        'price': price,
        'value': value
    }

# Example: Calculate your Apple position
position = calculate_position_value("AAPL", 100)

print("Position Calculation:")
print(f"  Stock:   {position['ticker']}")
print(f"  Shares:  {position['shares']}")
print(f"  Price:   ${position['price']:,.2f}")
print(f"  Value:   ${position['value']:,.2f}")
print(f"\nFormula: {position['shares']} shares × ${position['price']:,.2f} = ${position['value']:,.2f}")

Position Calculation:
  Stock:   AAPL
  Shares:  100
  Price:   $249.88
  Value:   $24,988.50

Formula: 100 shares × $249.88 = $24,988.50


## Portfolio: Collection of All Your Positions

A **portfolio** is all your positions combined.

**Portfolio Value Formula:**

$$\text{Portfolio Value} = \sum_{i=1}^{n} (\text{Shares}_i \times \text{Price}_i)$$



In [ ]:
import pandas as pd

def calculate_portfolio_value(holdings):
    """Calculate total portfolio value from holdings.

    Args:
        holdings: Dictionary of {ticker: shares}

    Returns:
        Dictionary with positions and total value
    """
    positions = []

    for ticker, shares in holdings.items():
        price = get_price(ticker)
        value = shares * price

        positions.append({
            'ticker': ticker,
            'shares': shares,
            'price': price,
            'value': value
        })

    total_value = sum(p['value'] for p in positions)

    return {
        'positions': positions,
        'total_value': total_value
    }

# Define a sample portfolio
my_portfolio = {
    'AAPL': 100,   # 100 shares of Apple
    'NVDA': 50,    # 50 shares of NVIDIA
    'MSFT': 75,    # 75 shares of Microsoft
    'GOOGL': 30    # 30 shares of Alphabet
}

# Calculate portfolio value
result = calculate_portfolio_value(my_portfolio)

# Display as a nice table
print("My Portfolio")
print("=" * 55)
print(f"{'Ticker':<8} {'Shares':>10} {'Price':>12} {'Value':>15}")
print("-" * 55)

for pos in result['positions']:
    print(f"{pos['ticker']:<8} {pos['shares']:>10,} ${pos['price']:>10,.2f} ${pos['value']:>13,.2f}")

print("-" * 55)
print(f"{'TOTAL':<8} {'':<10} {'':<12} ${result['total_value']:>13,.2f}")

My Portfolio
Ticker       Shares        Price           Value
-------------------------------------------------------
AAPL            100 $    249.84 $    24,984.00
NVDA             50 $    185.02 $     9,251.00
MSFT             75 $    447.29 $    33,546.38
GOOGL            30 $    331.64 $     9,949.20
-------------------------------------------------------
TOTAL                            $    77,730.58


## Cost Basis: What Did You Pay?

Your **cost basis** is what you originally paid for your shares.

**Cost Basis Formula:**

$$\text{Cost Basis} = \text{Shares} \times \text{Purchase Price}$$

**Why It Matters:**
1. **Profit/Loss Calculation:** Current Value - Cost Basis = Unrealized Gain/Loss
2. **Performance Tracking:** Did your investment decision pay off?

**Example:**
- You bought 100 AAPL at \$150. Cost Basis = \\$15,000
- Current price is \$175 → Current Value = \\$17,500
- Unrealized Gain = \$17,500 - \\$15,000 = \\$2,500 (16.7% return)

In [ ]:
def calculate_position_with_cost(ticker, shares, purchase_price):
    """Calculate position value including gain/loss.

    Args:
        ticker: Stock symbol
        shares: Number of shares
        purchase_price: Price per share when purchased
    """
    current_price = get_price(ticker)

    cost_basis = shares * purchase_price
    current_value = shares * current_price
    gain_loss = current_value - cost_basis
    return_pct = (gain_loss / cost_basis) * 100

    return {
        'ticker': ticker,
        'shares': shares,
        'purchase_price': purchase_price,
        'current_price': current_price,
        'cost_basis': cost_basis,
        'current_value': current_value,
        'gain_loss': gain_loss,
        'return_pct': return_pct
    }

# Example with cost basis
position = calculate_position_with_cost("AAPL", 100, 150.00)

print("Position Analysis with Cost Basis")
print("=" * 45)
print(f"Stock:           {position['ticker']}")
print(f"Shares:          {position['shares']}")
print(f"Purchase Price:  ${position['purchase_price']:,.2f}")
print(f"Current Price:   ${position['current_price']:,.2f}")
print("-" * 45)
print(f"Cost Basis:      ${position['cost_basis']:,.2f}")
print(f"Current Value:   ${position['current_value']:,.2f}")
print("-" * 45)

# Color-code the gain/loss
if position['gain_loss'] >= 0:
    print(f"Unrealized GAIN: ${position['gain_loss']:,.2f} (+{position['return_pct']:.1f}%)")
else:
    print(f"Unrealized LOSS: ${position['gain_loss']:,.2f} ({position['return_pct']:.1f}%)")

Position Analysis with Cost Basis
Stock:           AAPL
Shares:          100
Purchase Price:  $150.00
Current Price:   $249.79
---------------------------------------------
Cost Basis:      $15,000.00
Current Value:   $24,979.00
---------------------------------------------
Unrealized GAIN: $9,979.00 (+66.5%)


## Full Portfolio with Cost Basis

Let's track a complete portfolio with purchase prices to see overall performance.

In [ ]:
def analyze_portfolio(holdings_with_cost):
    """Analyze portfolio with cost basis tracking.

    Args:
        holdings_with_cost: List of dicts with ticker, shares, purchase_price
    """
    positions = []

    for holding in holdings_with_cost:
        pos = calculate_position_with_cost(
            holding['ticker'],
            holding['shares'],
            holding['purchase_price']
        )
        positions.append(pos)

    # Calculate totals
    total_cost = sum(p['cost_basis'] for p in positions)
    total_value = sum(p['current_value'] for p in positions)
    total_gain_loss = total_value - total_cost
    total_return_pct = (total_gain_loss / total_cost) * 100

    return {
        'positions': positions,
        'total_cost': total_cost,
        'total_value': total_value,
        'total_gain_loss': total_gain_loss,
        'total_return_pct': total_return_pct
    }

# Define portfolio with purchase prices
# (In real life, you'd track actual purchase dates and prices)
my_holdings = [
    {'ticker': 'AAPL', 'shares': 100, 'purchase_price': 150.00},
    {'ticker': 'NVDA', 'shares': 50, 'purchase_price': 450.00},
    {'ticker': 'MSFT', 'shares': 75, 'purchase_price': 320.00},
    {'ticker': 'GOOGL', 'shares': 30, 'purchase_price': 140.00}
]

# Analyze
analysis = analyze_portfolio(my_holdings)

# Display results
print("Portfolio Performance Analysis")
print("=" * 80)
print(f"{'Ticker':<8} {'Shares':>8} {'Cost':>12} {'Current':>12} {'Value':>12} {'Gain/Loss':>14}")
print("-" * 80)

for pos in analysis['positions']:
    gain_str = f"${pos['gain_loss']:+,.0f} ({pos['return_pct']:+.1f}%)"
    print(f"{pos['ticker']:<8} {pos['shares']:>8} ${pos['cost_basis']:>10,.0f} ${pos['current_value']:>10,.0f} {gain_str:>20}")

print("-" * 80)
total_str = f"${analysis['total_gain_loss']:+,.0f} ({analysis['total_return_pct']:+.1f}%)"
print(f"{'TOTAL':<8} {'':<8} ${analysis['total_cost']:>10,.0f} ${analysis['total_value']:>10,.0f} {total_str:>20}")

Portfolio Performance Analysis
Ticker     Shares         Cost      Current        Value      Gain/Loss
--------------------------------------------------------------------------------
AAPL          100 $    15,000 $    24,974     $+9,974 (+66.5%)
NVDA           50 $    22,500 $     9,251    $-13,249 (-58.9%)
MSFT           75 $    24,000 $    33,516     $+9,516 (+39.6%)
GOOGL          30 $     4,200 $     9,930    $+5,730 (+136.4%)
--------------------------------------------------------------------------------
TOTAL             $    65,700 $    77,671    $+11,971 (+18.2%)


## Real-Time Portfolio Tracker

Let's build a complete portfolio tracker that you can use with your own holdings.

In [ ]:
class PortfolioTracker:
    """A simple portfolio tracker for learning purposes."""

    def __init__(self):
        self.holdings = []

    def add_position(self, ticker, shares, purchase_price):
        """Add a position to the portfolio."""
        self.holdings.append({
            'ticker': ticker.upper(),
            'shares': shares,
            'purchase_price': purchase_price
        })
        print(f"Added: {shares} shares of {ticker} at ${purchase_price:.2f}")

    def get_current_value(self):
        """Calculate current portfolio value."""
        total = 0
        for h in self.holdings:
            price = get_price(h['ticker'])
            total += h['shares'] * price
        return total

    def get_cost_basis(self):
        """Calculate total cost basis."""
        return sum(h['shares'] * h['purchase_price'] for h in self.holdings)

    def summary(self):
        """Print portfolio summary."""
        print("\n" + "=" * 60)
        print("PORTFOLIO SUMMARY")
        print("=" * 60)

        total_cost = 0
        total_value = 0

        for h in self.holdings:
            current_price = get_price(h['ticker'])
            cost = h['shares'] * h['purchase_price']
            value = h['shares'] * current_price
            gain = value - cost

            total_cost += cost
            total_value += value

            print(f"\n{h['ticker']}:")
            print(f"  {h['shares']} shares @ ${h['purchase_price']:.2f} → ${current_price:.2f}")
            print(f"  Value: ${value:,.2f} (Gain/Loss: ${gain:+,.2f})")

        total_gain = total_value - total_cost
        total_return = (total_gain / total_cost) * 100 if total_cost > 0 else 0

        print("\n" + "-" * 60)
        print(f"Total Cost Basis:  ${total_cost:,.2f}")
        print(f"Total Value:       ${total_value:,.2f}")
        print(f"Total Gain/Loss:   ${total_gain:+,.2f} ({total_return:+.1f}%)")
        print("=" * 60)

# Create and use the tracker
tracker = PortfolioTracker()

# Add some positions
tracker.add_position("AAPL", 50, 165.00)
tracker.add_position("NVDA", 25, 500.00)
tracker.add_position("MSFT", 40, 350.00)

# Show summary
tracker.summary()

Added: 50 shares of AAPL at $165.00
Added: 25 shares of NVDA at $500.00
Added: 40 shares of MSFT at $350.00

PORTFOLIO SUMMARY

AAPL:
  50 shares @ $165.00 → $249.83
  Value: $12,491.50 (Gain/Loss: $+4,241.50)

NVDA:
  25 shares @ $500.00 → $184.96
  Value: $4,623.88 (Gain/Loss: $-7,876.12)

MSFT:
  40 shares @ $350.00 → $446.96
  Value: $17,878.40 (Gain/Loss: $+3,878.40)

------------------------------------------------------------
Total Cost Basis:  $34,750.00
Total Value:       $34,993.77
Total Gain/Loss:   $+243.77 (+0.7%)


## Key Takeaways

### Formulas You've Learned:

1. **Position Value** = Shares × Current Price

2. **Portfolio Value** = Sum of all Position Values

3. **Cost Basis** = Shares × Purchase Price

4. **Unrealized Gain/Loss** = Current Value - Cost Basis

5. **Return %** = (Gain/Loss ÷ Cost Basis) × 100

### Why This Matters for AI Agents:

When we build AI agents, they'll need to:
- Calculate position values to understand portfolio state
- Track cost basis to report performance
- Compare positions by value (not just share count)
- Identify which positions are winning or losing

**Next Notebook:** We'll learn about portfolio weights - understanding what percentage of your portfolio each position represents.